# Setup

In [ ]:
import os
import json
import re

import pandas as pd

from pathlib import Path

from src.code_processing import parse_code_string
from src.utils import split_users

CONFIG = {
    "version": "0.0.0",
    "only_correct": True,
    "only_final": True,
    "notes": "Final version of the dataset.",
}

DATA_PATH = Path("data")
IPYTHON_PATH = Path("data") / "ipython_new"
OUTPUT_PATH = DATA_PATH / "datasets" / ('ipython_' + CONFIG["version"])

***

# Loading and cleaning task information (items)

In [ ]:
# Load items data
items = pd.read_csv(IPYTHON_PATH / "item.csv", sep=";", index_col=0)

# Keep only relevant columns
items = items[["name", "instructions", "solution", "democode"]]

# Extract user instructions
items["instructions"] = items["instructions"].apply(lambda x: eval(x)[0][1])

# Extract and decode example solutions
items["solution"] = items["solution"].apply(lambda x: eval(x)[0][1]).apply(parse_code_string)
items["democode"] = items["democode"].apply(lambda x: eval(x)[0][1]).apply(parse_code_string)

# order by difficulty
ps_order = [
    "Proměnné a číselné výrazy",
    "Cyklus for",
    "Logické výrazy",
    "Podmíněný příkaz (if): základy",
    "Cyklus for s vnořenou podmínkou",
    "Cyklus while",
    "Podmíněný příkaz (if): těžší",
    "Posloupnosti",
    "Úpravy programů",
    "Řízení výpočtu",
    "Textové obrázky",
    "Řetězce: základy",
    "Seznamy",
    "Řetězce: těžší",
    "Seznamy a řetězce: vnořené",
    "Slovníky",
    "Záludné",
]

# load problem sets
ps = pd.read_csv(IPYTHON_PATH / "ps.csv", index_col=0, header=0, sep=";")
ps = ps[ps["url"].apply(lambda x: "interaktivni-python" in x)][["topic", "ordering"]]
ps.rename(columns={"ordering": "topic order"}, inplace=True)
ps["difficulty order"] = ps["topic"].apply(lambda x: ps_order.index(x))

# load mapping to items
item_to_ps = pd.read_csv(IPYTHON_PATH / "ps_problem.csv", index_col=0, header=0, sep=";")
item_to_ps = item_to_ps[
    item_to_ps["problem"].apply(lambda x: x in items.index) & item_to_ps["ps"].apply(lambda x: x in ps.index)
]

# add to items
keep = items.columns.union(ps.columns).tolist() + ["id"]
items = (
    items.reset_index(names="id")
    .merge(item_to_ps, left_on="id", right_on="problem")
    .merge(ps, left_on="ps", right_index=True)[keep]
)
items.set_index("id", inplace=True)

***

# Loading and cleaning submissions (log)

In [ ]:
# Load log data
log = pd.read_csv(IPYTHON_PATH / "log.csv", sep=";")

# Keep only relevant columns
log = log[["id", "user", "item", "answer", "correct", "responseTime", "time"]]

# Convert time column to datetime and correct data types
log["time"] = pd.to_datetime(log["time"])
log["correct"] = log["correct"].astype(bool)

# Decode submissions
log["answer"] = log["answer"].dropna().apply(parse_code_string)

# Sort by time
log.sort_values("time", inplace=True)

# Drop problematic rows
log.dropna(inplace=True)
log.drop_duplicates(inplace=True)

# Keep only submissions with at least 20 characters
log = log[log["answer"].map(len) >= 20]

# Keep only users with at least 5 submissions
log = log[log["user"].map(log["user"].value_counts()) >= 5]

# Keep only sessions lasting at least 1 minute
log = log[log["responseTime"] >= 6e4]

***

# Additional filtering

In [ ]:
# Filter out items with duplicate names
duplicate_names = items["name"].value_counts() > 1
duplicate_names = duplicate_names[duplicate_names].index
if len(duplicate_names) > 0:
    for item_name in duplicate_names:
        # keep only the first entry in items
        duplicate_entries = items[items["name"] == item_name].index
        shared_id = duplicate_entries[0]
        for id in duplicate_entries[1:]:
            # drop from items
            if id != shared_id:
                items = items[items.index != id]
            # propagate shared id to log
            log.loc[log["item"] == id, "item"] = shared_id

# Keep only items with at least 100 submissions
valid_items = items.index.isin((log["item"].value_counts() > 100).index)
items = items.loc[valid_items]

# Filter them out of the log also
log = log[log["item"].isin(items.index)]

***

# Loading and cleaning defect information (defects)

In [ ]:
# Load defects data
defects = pd.read_csv(DATA_PATH / "defects.csv")

# Keep only relevant columns
defects = defects[
    [
        "defect name",
        "EduLint code",
        "defect type",
        "description",
        "code example",
        "code fix example",
        "severity",
        "id",
    ]
]

# Drop defects not detected by EduLint
defects.dropna(subset=["EduLint code"], inplace=True)

# Convert EduLint codes from string to tuple
defects["EduLint code"] = defects["EduLint code"].apply(lambda x: tuple(map(str.strip, x.split(","))))

# Drop noisy defects
defects.drop([66, 4], axis=0, inplace=True)

# Create a dictionary mapping EduLint codes to defect indices
code_to_defect_id = {val: idx for idx, val in defects["EduLint code"].explode().items()}

# Replace missing values with empty strings
defects["code fix example"] = defects["code fix example"].fillna("")
defects["code example"] = defects["code example"].fillna("")

***

# Loading detected defects

In [ ]:
# Load message log data
messages = pd.read_csv(IPYTHON_PATH / "message_log.csv", index_col=0, header=0)

# Remove some of the messages associated with the "trailing whitespace" defect
messages = messages[~messages["message"].isin(["no newline at end of file", "trailing whitespace"])]

# Keep only the messages still in the ipython log
messages = messages[messages["log entry"].isin(log.index)]

# Keep only messages with an associated defect
messages = messages[messages["defect"].isin(code_to_defect_id.keys())]

# Use defect IDs instead of message codes
messages["defect"] = messages["defect"].replace(code_to_defect_id).astype(int)

messages.reset_index(drop=True, inplace=True)

# Create a defect log matrix
defect_log = pd.crosstab(messages["log entry"], messages["defect"]).reindex(log.index, fill_value=0)

# Keep only correct answers
if CONFIG["only_correct"]:
    log = log[log["correct"]]

# Keep only one answer per session
if CONFIG["only_final"]:
    log = log.reset_index().groupby(["user", "item"], as_index=False).last().set_index("index")

# Apply the filters to the defect log
defect_log = defect_log.loc[log.index]

# Keep only detected defects
encountered_defects = defect_log.sum() > 0
encountered_defects = encountered_defects[encountered_defects > 0]
defects = defects.loc[defects.index.isin(encountered_defects.index)]
defect_log = defect_log.loc[:, defects.index]

***
# Shortened names for plotting

In [ ]:
def truncate_text(text: str, max_length: int=20):
    """Truncate text if it exceeds the maximum length."""
    if len(text) > max_length:
        return text[:max_length - 3] + '...'
    return text

def abbreviate_and_truncate_text(text: str, ordered_abbreviations: dict | None=None, max_length: int=20):
    """Shorten text by applying a list of abbreviations and truncating if necessary."""
    current_text = text

    if ordered_abbreviations:
        for full_word, abbr in ordered_abbreviations.items():
            # Use regex with word boundaries to ensure we replace full words only
            pattern = re.escape(full_word)
            current_text = re.sub(pattern, abbr, current_text, flags=re.IGNORECASE)

            if len(current_text) <= max_length:
                return current_text
    
    return truncate_text(current_text)

ordered_abbreviations = {
    'whitespace': 'ws',
    'constant': 'const',
    'variable': 'var',
    'function': 'func',
    'parameter': 'param',
    'expression': 'expr',
    'argument': 'arg',
    'operator': 'op',
    'operation': 'op',
    'augmentable': 'aug',
    'assignment': 'assign',
    'container': 'cont',
    'statement': 'stmt',
    'arithmetic': 'arith',
    'condition': 'cond',
    'identifier': 'identif',
    'multiple': 'multi',
    'redundant': 'redun',
    'necessary': 'necces',
    'comparison': 'compar',
    'negated': 'neg',
    'unreachable': 'unreach',
    'inappropriate': 'inapp',
    'parenthesis': '()',
}

In [ ]:
defects['display name'] = defects['defect name'].apply(lambda x: abbreviate_and_truncate_text(x, ordered_abbreviations))
items['display name'] = items['name'].apply(truncate_text)

***
# Splitting to train, evaluation and test data

In [ ]:
development_mask, evaluation_mask, hold_out_mask = split_users(log, train_pct=0.8, val_pct=0.1, test_pct=0.1, seed=42)

***

# Combining to a final dataset

In [ ]:
os.makedirs(OUTPUT_PATH, exist_ok=True)

json.dump(CONFIG, open(OUTPUT_PATH / f"config_{CONFIG['version']}.json", "w"))

items.to_csv(OUTPUT_PATH / "items.csv")
defects.to_csv(OUTPUT_PATH / "defects.csv")

for dir, mask in zip(['development', 'evaluation', 'hold_out'], [development_mask, evaluation_mask, hold_out_mask]):
    path = OUTPUT_PATH / dir
    os.makedirs(path, exist_ok=True)
    log[mask].to_csv(path / "log.csv")
    defect_log[mask].to_csv(path / "defect_log.csv")

# multiple codes might map to one defect id
json.dump(code_to_defect_id, open(OUTPUT_PATH / "code_to_defect_id.json", "w"))